In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime

# Import the time module.
import time

# Import the API key.
from config import g_key

# Import the API key.
from config import weather_api_key

# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy

In [3]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/WeatherPy_challenge.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain Inches (last 3 hrs),Snow Inches (last 3 hrs)
0,0,Mataura,NZ,2020-01-05 06:15:09,-46.19,168.86,55.99,68,51,15.01,broken clouds,0.0,0.0
1,1,Vila Velha,BR,2020-01-05 06:15:09,-3.71,-38.60,86.00,78,20,4.70,few clouds,0.0,0.0
2,2,Nizhnekamsk,RU,2020-01-05 06:15:10,55.64,51.82,30.20,92,90,17.90,mist,0.0,0.0
3,3,Tuktoyaktuk,CA,2020-01-05 06:15:10,69.44,-133.03,-25.60,67,40,11.41,scattered clouds,0.0,0.0
4,4,New Norfolk,AU,2020-01-05 06:15:10,-42.78,147.06,69.01,42,0,17.22,clear sky,0.0,0.0


In [4]:
city_data_df.dtypes

City_ID                       int64
City                         object
Country                      object
Date                         object
Lat                         float64
Lng                         float64
Max Temp                    float64
Humidity                      int64
Cloudiness                    int64
Wind Speed                  float64
Current Description          object
Rain Inches (last 3 hrs)    float64
Snow Inches (last 3 hrs)    float64
dtype: object

In [75]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
rain_liketh = input("Do you want it to be raining? (yes/no) ")
snow_liketh = input("Do you want it to be snowing? (yes/no) ")

What is the minimum temperature you would like for your trip? 0
What is the maximum temperature you would like for your trip? 90
Do you want it to be raining? (yes/no) no
Do you want it to be snowing? (yes/no) yes


In [76]:
# Convert rain/snow responses to boolean
rain_liketh_bool = False
snow_liketh_bool = False

if rain_liketh == "yes":
    rain_liketh_bool = True
else:
    rain_liketh_bool = False

if snow_liketh == "yes":
    snow_liketh_bool = True
else:
    snow_liketh_bool = False

In [77]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp) & \
                                       (city_data_df["Rain Inches (last 3 hrs)"].astype('bool') == rain_liketh_bool) & \
                                       (city_data_df["Snow Inches (last 3 hrs)"].astype('bool') == snow_liketh_bool)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain Inches (last 3 hrs),Snow Inches (last 3 hrs)
17,17,Dikson,RU,2020-01-05 06:15:12,73.51,80.55,7.50,89,100,22.70,light snow,0.0,0.50
51,51,Tazovskiy,RU,2020-01-05 06:14:32,67.47,78.70,25.45,95,99,16.62,light snow,0.0,0.81
97,97,Severo-Kurilsk,RU,2020-01-05 06:15:32,50.68,156.12,17.76,90,100,6.15,light snow,0.0,0.25
117,117,Repnoye,RU,2020-01-05 06:15:36,51.59,43.18,34.00,91,100,5.99,overcast clouds,0.0,0.06
121,121,Velikiy Ustyug,RU,2020-01-05 06:15:37,60.76,46.30,22.84,96,88,5.82,light snow,0.0,0.13
133,133,Vytegra,RU,2020-01-05 06:15:40,61.01,36.45,30.25,92,99,8.19,light snow,0.0,0.38
153,153,Yerbogachen,RU,2020-01-05 06:15:44,61.28,108.01,14.09,90,100,12.28,light snow,0.0,0.69
157,157,Ostrovnoy,RU,2020-01-05 06:15:45,68.05,39.51,15.73,85,100,25.30,light snow,0.0,1.13
232,232,Teguldet,RU,2020-01-05 06:16:02,57.31,88.17,24.03,92,100,11.48,light snow,0.0,0.50
244,244,Berlevag,NO,2020-01-05 06:16:06,70.86,29.09,28.99,62,100,33.02,light snow,0.0,0.31


In [78]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [86]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Current Description", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""

In [90]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to the params dictionary as values to the location key.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [92]:
hotel_df.head()

,City,Country,Current Description,Max Temp,Lat,Lng,Hotel Name
17,Dikson,RU,light snow,7.50,73.51,80.55,
51,Tazovskiy,RU,light snow,25.45,67.47,78.70,TazHostel
97,Severo-Kurilsk,RU,light snow,17.76,50.68,156.12,Kak Doma
117,Repnoye,RU,overcast clouds,34.00,51.59,43.18,"ООО ЛОЦ ""Современная медицина"""
121,Velikiy Ustyug,RU,light snow,22.84,60.76,46.30,Dvina


In [99]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} {Max Temp} °F</dd>
</dl>
"""

# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [109]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
cur_desc = hotel_df["Current Description"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Save map
plt.savefig('weather_data/WeatherPy_vacation_map.png', dpi=300, bbox_inches='tight')

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

In [103]:
# Create the output file (CSV).
output_data_file = "weather_data/WeatherPy_vacation.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")